In [1]:
! git clone https://github.com/Nelson-Gon/cytounet.git

Cloning into 'cytounet'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 828 (delta 58), reused 66 (delta 26), pack-reused 726
Receiving objects: 100% (828/828), 49.86 MiB | 36.19 MiB/s, done.
Resolving deltas: 100% (332/332), done.


In [2]:
%cd cytounet

/kaggle/working/cytounet


In [3]:
from cytounet.model import *
from cytounet.augmentation import *
from cytounet.data import *

Using TensorFlow backend.


In [4]:
data_gen_args = dict(rotation_range=0.05,
                      rescale = 1./255,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

In [5]:
! if [ ! -d "aug" ]; then mkdir aug aug/images aug/masks;fi

In [6]:
my_generator = generate_train_data(16,'examples/legacy/data/train','image','label',data_gen_args,save_to_dir = None, seed= 25,
                              target_size = (256, 256))

In [7]:
# save space on device
#! rm -rf aug

In [8]:
model = unet(input_size = (256, 256, 1),optimiser='Adam', metrics=['accuracy'], loss='binary_crossentropy', 
             learning_rate=3e-4) 

In [9]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

ImportError: cannot import name 'ReduceLRonPlateau' from 'keras.callbacks' (/opt/conda/lib/python3.7/site-packages/keras/callbacks/__init__.py)

In [ ]:
history = model.fit_generator(my_generator,steps_per_epoch= 800,epochs=5, 
                              callbacks = [EarlyStopping(verbose=1, monitor='loss', patience = 2),
                                          ReduceLROnPlateau(verbose=1, monitor='loss', patience =2)])

In [ ]:
model.save("unet_segment.hdf5")

In [ ]:
test_generator = generate_test_data("examples/legacy/data/test", 30, image_suffix="png")

In [ ]:
model = unet(pretrained_weights="unet_segment.hdf5")
results = model.predict_generator(test_generator,30,verbose=1)

In [ ]:
show_images(results, number = 10)